<a href="https://colab.research.google.com/github/AhmadAkbariR/PI-DeepONet/blob/main/2D%20elasticity%20DeepONet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @ 2D elastic square domain --> PINN-DeepONet

"""Backend supported: tensorflow, pytorch, paddle"""

import os
os.environ['DDE_BACKEND'] = 'pytorch'
try:
   import deepxde as dde
except ImportError:
   !pip install deepxde

#import deepxde as dde
import matplotlib.pyplot as plt
import numpy as np
current_backend = dde.backend.backend_name
print("Current default backend is:", current_backend)

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
# @title DeepONet for elasticity
# PDE equation
def pde(xy, uv, aux):
    mu = 0.01
    nu = 0.3
    u, v = uv[..., 0:1], uv[..., 1:2]
    #u, v, p = uvp[..., 0:1], uvp[..., 1:2], uvp[..., 2:3]
    grad_u = dde.zcs.LazyGrad(xy, u)
    grad_v = dde.zcs.LazyGrad(xy, v)
    #grad_p = dde.zcs.LazyGrad(xy, p)
    # first order
    du_x = grad_u.compute((1, 0))
    dv_y = grad_v.compute((0, 1))

    grad_du_x = dde.zcs.LazyGrad(xy, du_x)
    grad_dv_y = dde.zcs.LazyGrad(xy, dv_y)

    #dp_x = grad_p.compute((1, 0))
    #dp_y = grad_p.compute((0, 1))
    # second order
    du_xx = grad_u.compute((2, 0))
    du_yy = grad_u.compute((0, 2))
    du_xy = grad_du_x.compute((0, 1))
    dv_xx = grad_v.compute((2, 0))
    dv_xy = grad_dv_y.compute((1, 0))
    dv_yy = grad_v.compute((0, 2))
    Del2U = du_xx + du_yy + (1)/(1-2*nu) * (du_xx + dv_xy)
    Del2V = dv_xx + dv_yy + (1)/(1-2*nu) * (du_xy + dv_yy)

    #motion_x = mu * (du_xx + du_yy) - dp_x
    #motion_y = mu * (dv_xx + dv_yy) - dp_y
    #mass = du_x + dv_y
    #return motion_x, motion_y, mass
    return Del2U, Del2V

# Geometry
lx = 1.0
ly = 0.5
geom = dde.geometry.Rectangle([0, 0], [lx, ly])

# Boundary condition
# other boundary conditions will be enforced by output transform

# Dirichlet bc on the right edge- U direction
def bc_right_U_func(x, aux_var):
    return  dde.backend.as_tensor( 1.  )  # uniform U =1 displacement

bc_right_U = dde.icbc.DirichletBC(
                                    geom=geom,
                                    func=bc_right_U_func,
                                    on_boundary=lambda x, on_boundary: np.isclose(x[0], lx),
                                    component= 0 ) # component = 0 means x direction

# Dirichlet bc on the right edge- V direction
# def bc_right_V_func(x, aux_var):
#     return  dde.backend.as_tensor( 0. )
#bc_right_V = dde.icbc.DirichletBC(
#                                    geom=geom,
#                                    func=bc_right_V_func,
#                                    on_boundary=lambda x, on_boundary: np.isclose(x[0], 1.),
#                                    component= 1)
#def bc_slip_top_func(x, aux_var):
    # using (perturbation / 10 + 1) * x * (1 - x)
#    return (aux_var / 10 + 1.) * dde.backend.as_tensor(x[:, 0:1] * (1 - x[:, 0:1]))
#bc_slip_top = dde.icbc.DirichletBC(
#                                    geom=geom,
#                                    func=bc_slip_top_func,
#                                    on_boundary=lambda x, on_boundary: np.isclose(x[1], 1.),
#                                    component=0)

# PDE object
# Define the boundary condition functions

def boundary_left(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

def boundary_right(x, on_boundary):
    return on_boundary and np.isclose(x[0], lx)

def boundary_top(x, on_boundary):
    return on_boundary and np.isclose(x[1], ly)

def boundary_bottom(x, on_boundary):
    return on_boundary and np.isclose(x[1], 0)

def First_grad(x, u):
    u_grad_x = dde.grad.jacobian(u, x, i=0, j=0)  # ∂u/∂x
    v_grad_y = dde.grad.jacobian(u, x, i=1, j=1)  # ∂v/∂y
    u_grad_y = dde.grad.jacobian(u, x, i=0, j=1)  # ∂u/∂x
    v_grad_x = dde.grad.jacobian(u, x, i=1, j=0)  # ∂u/∂x
    return u_grad_x, v_grad_y, u_grad_y, v_grad_x

def Right_constraint(x , u, X=[]):
    # Compute the gradients
    u_grad_y = dde.grad.jacobian(u, x, i=0, j=1)  # ∂u/∂x
    v_grad_x = dde.grad.jacobian(u, x, i=1, j=0)  # ∂u/∂x
    # The Neumann condition: 2*nu*∂u/∂x + ∂v/∂y = 0
    return  u_grad_y + v_grad_x

def Bottom_constraint(x , u, X=[]):
    # Compute the gradients
    nu = 0.3
    # The Neumann condition: 2*nu*∂u/∂x + ∂v/∂y = 0
    u_grad_x = dde.grad.jacobian(u, x, i=0, j=0)  # ∂u/∂x
    v_grad_y = dde.grad.jacobian(u, x, i=1, j=1)  # ∂v/∂y
    return  (1-nu)*v_grad_y + nu*u_grad_x

def top_contraint1(x , u, X=[]):
    # Compute the gradients
    u_grad_y = dde.grad.jacobian(u, x, i=0, j=1)  # ∂u/∂x
    v_grad_x = dde.grad.jacobian(u, x, i=1, j=0)  # ∂u/∂x
    # The Neumann condition: 2*nu*∂u/∂x + ∂v/∂y = 0
    return u_grad_y + v_grad_x

def top_contraint2(x , u, X=[]):
    # Compute the gradients
    u_grad_x = dde.grad.jacobian(u, x, i=0, j=0)  # ∂u/∂x
    v_grad_y = dde.grad.jacobian(u, x, i=1, j=1)  # ∂v/∂y
    nu = 0.3
    # The Neumann condition: 2*nu*∂u/∂x + ∂v/∂y = 0
    return  (1-nu)*v_grad_y + nu*u_grad_x


bc_right_cstrit = dde. OperatorBC(geom, Right_constraint, boundary_right)  # u(x=1, y) = 1
bc_bottom_cstrit = dde. OperatorBC(geom, Bottom_constraint, boundary_bottom)  # u(x=1, y) = 1
bc_top_cstrit1 = dde. OperatorBC(geom, top_contraint1, boundary_top)  # u(x=1, y) = 1
bc_top_cstrit2 = dde. OperatorBC(geom, top_contraint2, boundary_top)  # u(x=1, y) = 1
#bc4 = dde.DirichletBC(geom, lambda x: 0, boundary_bottom, component=1)  # v(x, y=0) = 0
#bc5 = dde.NeumannBC(geom, lambda x: 0, boundary_bottom, component=1)  #
#bc6 = dde.NeumannBC(geom, lambda x: 0, boundary_top, component=0)  #
#bc7 = dde.NeumannBC(geom, lambda x: 0, boundary_top, component=1)  #
# Define the Neumann boundary condition

pde_obj = dde.data.PDE(
    geom,
    pde,
    bcs=[bc_right_U, bc_right_cstrit, bc_bottom_cstrit, bc_top_cstrit1, bc_top_cstrit2],
    num_domain=500,
    num_boundary=50,  # sampling a bit more points on boundary (1000 on top bc)
    num_test=100,
)

# Function space
func_space = dde.data.GRF(length_scale=0.2)

# Data
n_pts_edge = 101  # using the size of true solution, but this is unnecessary
eval_pts = np.linspace(0, ly, num=n_pts_edge)[:, None]
data = dde.zcs.PDEOperatorCartesianProd(
    pde_obj, func_space, eval_pts, num_function=100,
    function_variables=[0], num_test=20, batch_size=10
)

# Net
net = dde.nn.DeepONetCartesianProd(
    [n_pts_edge, 50, 50, 50],
    [2, 50, 50, 50],
    "tanh",
    "Glorot normal",
    num_outputs=2,
    multi_output_strategy="independent"
)

# Output transform for zero boundary conditions
def out_transform(inputs, outputs):
    x, y = inputs[1][:, 0], inputs[1][:, 1]
    # horizontal displacement on left
    u = outputs[:, :, 0] * (x)[None, :]
    # vertical dis on left
    v = outputs[:, :, 1] * (x)[None, :] * (y)[None, :]
    ## pressure on bottom
    #p = outputs[:, :, 2] * y[None, :]
    return dde.backend.stack((u, v), axis=2)
net.apply_output_transform(out_transform)

# Model
model = dde.zcs.Model(data, net)
model.compile("adam", lr=0.001, decay=("inverse time", 1000, 0.5))
losshistory, train_state = model.train(iterations=10000)


dde.utils.plot_loss_history(losshistory)
plt.show()

# save model if needed
# model.save('stokes_weights')

# Evaluation
func_feats = func_space.random(10)
v = func_space.eval_batch(func_feats, eval_pts)
v[:] = 0.  # true solution uses zero perturbation
xv, yv = np.meshgrid(eval_pts[:, 0], eval_pts[:, 0], indexing='ij')
xy = np.vstack((np.ravel(xv), np.ravel(yv))).T
sol_pred = model.predict((v, xy))[0]
#sol_true = np.load('../dataset/stokes.npz')['arr_0']
#print('Error on horizontal velocity:', dde.metrics.l2_relative_error(sol_true[:, 0], sol_pred[:, 0]))
#print('Error on vertical velocity:', dde.metrics.l2_relative_error(sol_true[:, 1], sol_pred[:, 1]))
#print('Error on pressure:', dde.metrics.l2_relative_error(sol_true[:, 2], sol_pred[:, 2]))
plt.figure(2)
def deform_coord(xy , sol_pred):
    return xy + sol_pred
deformed = deform_coord(xy , sol_pred)
plt.scatter(deformed[:,0],deformed[:,1])
plt.show()


# PLOT RESULTS

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming X_test contains original coordinates and predictions contains the predicted coordinates after deformation
# Get the predictions
#predictions = model.predict(X_test)

# Deformed coordinates
deformed = deform_coord(X_test, predictions)

# Calculate displacements in the x direction
u = predictions[:, 0]
v = predictions[:, 1]

# Plot the deformed shape with color map for the displacement in x direction
plt.figure()

# Plot undeformed shape boundary

plot_boundary(X_test, 'Undeformed Shape')

scatter = plt.scatter(deformed[:, 0], deformed[:, 1], s = 1, c=u, cmap='jet', vmin=0, vmax=1)
plt.colorbar(scatter, label='Displacement in X direction')
plt.title(r'$u$')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.axis('equal')  # To maintain aspect ratio
plt.show()

# Plot the deformed shape with color map for the displacement in y direction
plt.figure()
scatter = plt.scatter(deformed[:, 0], deformed[:, 1], s = 1, c=v, cmap='jet', vmin=min(v), vmax=max(v))
plt.colorbar(scatter, label='Displacement in Y direction')
plt.title(r'$v$')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.axis('equal')  # To maintain aspect ratio
plt.show()

# Plot the deformed shape with color map for the s_xx
plt.figure()
scatter = plt.scatter(deformed[:, 0], deformed[:, 1], s = 1, c=sigma_x, cmap='jet', vmin=sigma_x.min(), vmax=sigma_x.max())
plt.colorbar(scatter, label='Stress,'r'$\sigma_{xx}$')
plt.title(r'$\sigma_{xx}$')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.axis('equal')  # To maintain aspect ratio
plt.show()


# Plot the deformed shape with color map for the s_yy
plt.figure()
scatter = plt.scatter(deformed[:, 0], deformed[:, 1], s = 1, c=sigma_y, cmap='jet', vmin=min(sigma_y), vmax=max(sigma_y))
plt.colorbar(scatter, label='Stress, 'r'$\sigma_{yy}$')
plt.title(r'$\sigma_{yy}$')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.axis('equal')  # To maintain aspect ratio
plt.show()

# Plot the deformed shape with color map for the s_xy
plt.figure()
scatter = plt.scatter(deformed[:, 0], deformed[:, 1], s = 1, c=sigma_xy, cmap='jet', vmin=min(sigma_xy), vmax=max(sigma_xy))
plt.colorbar(scatter, label= r'$\sigma_{xy}$')
plt.title(r'$\sigma_{xy}$')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

plt.axis('equal')  # To maintain aspect ratio
plt.show()

In [ ]:
# @title plot deformation
def Deformed_shape(xy, uv):
    x = xy[:,0]  # Original x coordinates
    y = xy[:,1]  # Original y coordinates
    u = uv[:,0]  # Random displacement in x direction
    v = uv[:,1]  # Random displacement in y direction


    # New coordinates after displacement

    x_new = x + u
    y_new = y + v

    # Plotting
    plt.figure(figsize=(12, 8))
    plt.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=1, color='blue', alpha=0.5, label='Displacement (u, v)')
    plt.scatter(x, y, color='red', s=10, alpha=0.7, label='Original Nodes')
    plt.scatter(x_new, y_new, color='green', s=10, alpha=0.7, label='New Nodes')

    # Setting the limits for better visualization
    plt.xlim(-0.1, 2.1)  # Adjust the range based on your data
    plt.ylim(-0.1, 1.1)
    plt.grid()
    plt.axhline(0, color='grey', lw=0.5)
    plt.axvline(0, color='grey', lw=0.5)
    plt.title('Original and Displaced Nodes')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.legend()
    plt.show()

Deformed_shape(xy, sol_pred)